In [2]:
import pandas as pd
# import seaborn


pd.set_option('display.max_colwidth', 355)
pd.set_option('display.max_rows', 100)

/Users/rodrigofaria/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
df = pd.read_csv('datasets/b3_stocks_1994_2020.csv')

### Licitações Janeiro 2021 - Exploração inicial

In [7]:
df.head()

,datetime,ticker,open,close,high,low,volume
0,1994-07-04,ACE 3,48.00,48.00,48.00,47.00,46550.0
1,1994-07-04,ALP 3,155.27,156.00,156.00,155.27,163405.8
2,1994-07-04,ALP 4,131.00,131.00,131.00,131.00,6550.0
3,1994-07-04,IBP 6,600.00,600.00,600.00,600.00,7800.0
4,1994-07-04,AQT 4,0.89,0.99,0.99,0.85,13137.0


In [8]:
df.shape

(1883203, 7)

In [12]:
df['delta_open'] = df.apply(lambda x: abs(x['open'] - x['close']), axis=1)

In [16]:
df['delta_high'] = df.apply(lambda x: abs(x['high'] - x['low']), axis=1)

In [10]:
import matplotlib

ModuleNotFoundError: No module named 'matplotlib'

### Items de Licitações Janeiro 2021 - Exploração inicial

In [5]:
df_items = pd.read_csv('202101_Licitacoes/202101_ItemLicitacao.csv',  encoding='latin', error_bad_lines=False, sep=';')

/Users/rodrigofaria/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df_items.head()

,Número Licitação,Código UG,Nome UG,Código Modalidade Compra,Modalidade Compra,Número Processo,Código Órgão,Nome Órgão,Código Item Compra,Descrição,Quantidade Item,Valor Item,CNPJ Vencedor,Nome Vencedor
0,12019,154628,INST.FEDERAL FARROUPILHA CAMPUS JAGUARI,2,Tomada de Preços,26420000767201908,26420,Instituto Federal Farroupilha,1546280200001201900001,ESTUDOS E PROJETOS DE ARQUITETURA,3884,"14784,8500",08427697000193,EXTINCHAMA EXTINTORES BR 287 LTDA;
1,12020,927843,SECRETARIA DE EST.DE PLANEJ.E ADMINISTRAÇÃO,-99,Pregão - Registro de Preço,2019/204734,93420,ESTADO DO PARA,9278430500001202000001,TINTA PARA CARIMBO,10806,"32418,0000",31920842000195,MM COMERCIO ATACADISTA DE ARTIGOS DE PAPELARIA E SERVICOS LTDA
2,12020,927843,SECRETARIA DE EST.DE PLANEJ.E ADMINISTRAÇÃO,-99,Pregão - Registro de Preço,2019/204734,93420,ESTADO DO PARA,9278430500001202000002,COMPONENTES CARIMBO,2697,"59334,0000",31920842000195,MM COMERCIO ATACADISTA DE ARTIGOS DE PAPELARIA E SERVICOS LTDA
3,12020,927843,SECRETARIA DE EST.DE PLANEJ.E ADMINISTRAÇÃO,-99,Pregão - Registro de Preço,2019/204734,93420,ESTADO DO PARA,9278430500001202000003,COMPONENTES CARIMBO,4120,"45320,0000",31920842000195,MM COMERCIO ATACADISTA DE ARTIGOS DE PAPELARIA E SERVICOS LTDA
4,12020,927843,SECRETARIA DE EST.DE PLANEJ.E ADMINISTRAÇÃO,-99,Pregão - Registro de Preço,2019/204734,93420,ESTADO DO PARA,9278430500001202000004,COMPONENTES CARIMBO,2740,"52279,2000",31920842000195,MM COMERCIO ATACADISTA DE ARTIGOS DE PAPELARIA E SERVICOS LTDA


In [7]:
df_items.columns

Index(['Número Licitação', 'Código UG', 'Nome UG', 'Código Modalidade Compra',
       'Modalidade Compra', 'Número Processo', 'Código Órgão', 'Nome Órgão',
       'Código Item Compra', 'Descrição', 'Quantidade Item', 'Valor Item',
       'CNPJ Vencedor', 'Nome Vencedor'],
      dtype='object')

In [8]:
df_items.shape

(47676, 14)

Número de items por licitação

In [9]:
df_items['Número Licitação'].value_counts()

12021       3574
22020       2752
42020       1817
52020       1575
12020       1469
            ... 
10482020       1
1162021        1
11812020       1
12692020       1
12019          1
Name: Número Licitação, Length: 637, dtype: int64

In [10]:
df_items.loc[(df_items['Número Licitação'] == 22020) & (df_items['Código UG'] == 926418)].head()

,Número Licitação,Código UG,Nome UG,Código Modalidade Compra,Modalidade Compra,Número Processo,Código Órgão,Nome Órgão,Código Item Compra,Descrição,Quantidade Item,Valor Item,CNPJ Vencedor,Nome Vencedor
5045,22020,926418,COMPANHIA DE SERVIÇOS URBANOS DE NATAL,-99,Pregão - Registro de Preço,20200439039,94420,ESTADO DO RIO GRANDE DO NORTE,9264180500002202000001,CAMISA UNIFORME,5000,"81250,0000",35200562000163,JESSICA LORENA CUNHA SILVA 00871015994
5046,22020,926418,COMPANHIA DE SERVIÇOS URBANOS DE NATAL,-99,Pregão - Registro de Preço,20200439039,94420,ESTADO DO RIO GRANDE DO NORTE,9264180500002202000002,CALÇA,5000,"86500,0000",02440676000121,COMERCIAL APOLO LTDA
5047,22020,926418,COMPANHIA DE SERVIÇOS URBANOS DE NATAL,-99,Pregão - Registro de Preço,20200439039,94420,ESTADO DO RIO GRANDE DO NORTE,9264180500002202000003,CHAPÉU MASCULINO,2500,"27500,0000",02440676000121,COMERCIAL APOLO LTDA
5048,22020,926418,COMPANHIA DE SERVIÇOS URBANOS DE NATAL,-99,Pregão - Registro de Preço,20200439039,94420,ESTADO DO RIO GRANDE DO NORTE,9264180500002202000004,CALÇA,800,"23200,0000",02440676000121,COMERCIAL APOLO LTDA
5049,22020,926418,COMPANHIA DE SERVIÇOS URBANOS DE NATAL,-99,Pregão - Registro de Preço,20200439039,94420,ESTADO DO RIO GRANDE DO NORTE,9264180500002202000005,MACACÃO,100,"7200,0000",02440676000121,COMERCIAL APOLO LTDA


In [11]:
df_jan.loc[df_jan['Número Licitação'] == 22020].head()

,Número Licitação,Código UG,Nome UG,Código Modalidade Compra,Modalidade Compra,Número Processo,Objeto,Situação Licitação,Código Órgão Superior,Nome Órgão Superior,Código Órgão,Nome Órgão,Município,Data Resultado Compra,Data Abertura,Valor Licitação
693,22020,925016,FUNDAÇÃO MUNICIPAL DE SAÚDE DE NITERÓI/RJ,-99,Pregão - Registro de Preço,200009061/2019,"Objeto: Pregão Eletrônico - Eventual fornecimento de água mineral natural (sem gás), acondicionada em garrafão de 20 (vinte) litros",Divulgado,99900,REPUBLICA FEDERATIVA DO BRASIL,95320,ESTADO DO RIO DE JANEIRO,Inválido,07/01/2021,20/02/2020,"148552,8000"
694,22020,926418,COMPANHIA DE SERVIÇOS URBANOS DE NATAL,-99,Pregão - Registro de Preço,20200439039,Objeto: Pregão Eletrônico - Aquisição de Fardamento,Divulgado,99900,REPUBLICA FEDERATIVA DO BRASIL,94420,ESTADO DO RIO GRANDE DO NORTE,Inválido,21/01/2021,30/12/2020,"840,0000"
695,22020,925894,COMPANHIA DE SANEAMENTO MUNICIPAL,5,Pregão,0022020,"Objeto: Pregão Eletrônico - Contratação de empresa especializada em CallCenter nas modalidades de tele atendimento (ativo e receptivo) na forma humana e eletrônica, envio de mensagens de texto via SMS, atendimento via e-mail (fale conosco), aplicativos de mensagem instantânea (whatsapp) e telecobrança.",Evento de Alteração Divulgado,99900,REPUBLICA FEDERATIVA DO BRASIL,95120,ESTADO DE MINAS GERAIS,Inválido,15/01/2021,25/09/2020,"1618990,2600"
696,22020,925445,COMPANHIA DE HABITAÇÃO DO ESTADO DO PARÁ,5,Pregão,2020/464355,"Objeto: Pregão Eletrônico - Contratação de empresa especializada para execução dos serviços de Auditoria Independente sobre as Demonstrações Contábeis da Companhia de Habitação do Estado do Pará COHAB/PA, com emissão de Relatórios Circunstanciados e Parecer de Auditoria sobre as análises procedidas, na forma exigida pela legislação vigente, confo...",Divulgado,99900,REPUBLICA FEDERATIVA DO BRASIL,93420,ESTADO DO PARA,Inválido,07/01/2021,28/12/2020,"19800,0000"
697,22020,130017,LABORATORIO FEDERAL DE DEFESA AGROPECUARIA/PA,-99,Pregão - Registro de Preço,21003.000336/2020,"Objeto: Pregão Eletrônico - Aquisição de materiais químicos e materiais de referência certificados, para atendimento prioritariamente das Unidades Laboratoriais que integram a Divisão Técnica Laboratorial do Laboratório Federal de Defesa Agropecuária no Pará (DLAB/LFDA-PA), conforme condições, quantidades e exigências estabelecidas.",Publicado,22000,"Ministério da Agricultura, Pecuária e Abastec",22000,"Ministério da Agricultura, Pecuária e Abastec",BELEM,15/01/2021,18/12/2020,"990,0000"


In [19]:
pd.DataFrame(df_items['Descrição'].value_counts()).head(100)

,Descrição
FORNECIMENTO / TRANSPORTE DE AGUA - CARRO PIPA,3928
SUPERVISAO / GERENCIAMENTO / FISCALIZACAO - PROJETO CONSTRU-CAO / OBRAS CIVIS,924
MANUTENCAO DE VEICULOS LEVES / PESADOS,821
CONEXÃO HIDRÁULICA,460
TRANSPORTE RODOVIARIO - CARGAS / ENCOMENDAS,454
REAGENTE PARA DIAGNÓSTICO CLÍNICO 5,414
FIO DE SUTURA,386
AR CONDICIONADO - MANUTENCAO DE APARELHOS DE PAREDE,306
BROCA ALTA ROTAÇÃO,276
Informação protegida por sigilo nos termos da legislação vigente,266


In [17]:
pd.DataFrame(df_items['Descrição'].value_counts()).shape

(4925, 1)

In [28]:
df_fita = df_items.loc[df_items['Descrição'] == 'FITA ADESIVA', ['Valor Item', 'Quantidade Item']]
df_fita.head()

,Valor Item,Quantidade Item
69,"14277,6000",23796
70,"22052,8500",27915
71,"152148,9600",30552
72,"40698,0000",34200
73,"97218,0000",44190


In [29]:
preco_fita = []
for index, row in df_fita.iterrows():
    preco_fita.append(float(row['Valor Item']) / float(row['Quantidade Item']))
print(preco_fita)

ValueError: could not convert string to float: '14277,6000'